In [ ]:
import persistence_homology as ph

In [ ]:
import csv
import os
import json
from pathlib import Path
import logging

In [ ]:
log_path = Path("notebook_demo.log")
logging.basicConfig(
    filename='automation.log',         # log file path
    filemode='a',                  # 'w' to overwrite each run, 'a' to append
    format='%(asctime)s %(levelname)-8s %(name)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO             # minimum level to record
)
logger = logging.getLogger(__name__)

logger.info("Notebook logging started")

In [ ]:
def read_csv(path):
    with open(path, newline="") as f:
        reader = csv.reader(f)
        output = []
        for row in reader:
            values = []
            for value in row:
                if len(row) == 3:
                    values.append(float(value))
                elif len(row) == 2:
                    values.append(int(value))
            output.append(values)
        return output
def get_subfolders(path):
    """
    Return a list of names of all subdirectories in the given path.
    """
    return [
        name for name in os.listdir(path)
        if os.path.isdir(os.path.join(path, name))
    ]
def write_json(json_file_name, data):
    output_path = Path("{}.json".format(json_file_name))
    with output_path.open("w", encoding="utf-8") as f:
        json.dump(
            data,            # Python object
            f,               # file handle
            ensure_ascii=False,  # keep unicode chars if any
            indent=4             # pretty-print with 4-space indent
        )

In [ ]:
segmentation_folder_path = "./lung_segmentations"
lung_segmentations = get_subfolders(segmentation_folder_path)

# Filtration Algorithm Start

In [ ]:
json_data = []
for lung_segmentation in lung_segmentations:
    folder_path = './lung_segmentations/{}'.format(lung_segmentation)
    edges_path = '/edges.csv'
    vertices_path = '/vertices.csv'
    vertices = read_csv(folder_path + vertices_path)
    edges = read_csv(folder_path + edges_path)
    directions = ph.generate_sphere_points(20, 5, 0.0000001)
    directions = [[0,0,1]]
    print("Lung Segmentation-{}:".format(lung_segmentation))
    for direction in directions:
        bz = ph.BettiZero(direction, vertices, edges)
        components, mergers, verts, births = bz.compute_persistence()
        intervals = ph.compute_intervals(births, mergers)
        largest_bar = ph.compute_largest_bar(intervals)
        data = {
            "segmentation": lung_segmentation,
            "direction": direction,
            "intervals": intervals,
            "largest_bar": largest_bar,
            "components": components.keys()
        }
        json_data.append(data)
        print("Direction: {}. Largest Bar: {}.".format(direction, largest_bar))
        logger.info(data)
write_json("BettiZeroSegmentations", json_data)